In [10]:
import time
import numpy as np
import torch
import warnings
import torch.nn as nn
from typing import Tuple
import numpy as np
from robustbench.data import get_preprocessing, load_clean_dataset
from robustbench.model_zoo.enums import BenchmarkDataset, ThreatModel
from robustbench.utils import clean_accuracy
from pathlib import Path
from typing import Callable, Optional, Sequence, Tuple, Union
from autoattack import AutoAttack

In [11]:
def benchmark(
    model: Union[nn.Module, Sequence[nn.Module]],
    n_examples: int = 10000,
    dataset: Union[str, BenchmarkDataset] = BenchmarkDataset.cifar_10,
    threat_model: Union[str, ThreatModel] = ThreatModel.Linf,
    model_name: Optional[str] = None,
    data_dir: str = "./data",
    device: Optional[Union[torch.device, Sequence[torch.device]]] = None,
    batch_size: int = 32,
    eps: Optional[float] = None,
    log_path: Optional[str] = None,

    version='custom', #'rand', # 'rand' for models using stochasticity

    preprocessing: Optional[Union[str, Callable]] = None,
    aa_state_path: Optional[Path] = None,

    ) -> Tuple[float, float]:
    if isinstance(model, Sequence) or isinstance(device, Sequence):
        # Multiple models evaluation in parallel not yet implemented
        raise NotImplementedError

    try:
        if model.training:
            warnings.warn(Warning("The given model is *not* in eval mode."))
    except AttributeError:
        warnings.warn(
            Warning(
                "It is not possible to asses if the model is in eval mode"))

    dataset_: BenchmarkDataset = BenchmarkDataset(dataset)
    threat_model_: ThreatModel = ThreatModel(threat_model)
    model = model.cuda()

    prepr = get_preprocessing(dataset_, threat_model_, model_name,
                            preprocessing)

    clean_x_test, clean_y_test = load_clean_dataset(dataset_, n_examples,
                                                    data_dir, prepr)

    clean_accs = []
    for i in range(10):
        accuracy = clean_accuracy(model,
                                clean_x_test,
                                clean_y_test,
                                batch_size=batch_size,
                                device=device)
        clean_accs.append(accuracy)
    print(f'With {len(clean_x_test)} clean examples, test for 10 times, clean accuracy: {np.mean(clean_accs):.2%}±{np.std(clean_accs):.2%}')

    adversary = AutoAttack(
                        model,  # at default, this will call aggreegated model
                        norm=threat_model_.value,
                        eps=eps,
                        version=version,
                        device=device,
                        log_path=log_path,
                        attacks_to_run=['apgd-ce', 'apgd-t'] if version == "custom" else [], #Stan's addition
                        )
    x_adv = adversary.run_standard_evaluation(
                                            clean_x_test,
                                            clean_y_test,
                                            bs=batch_size,
                                            state_path=aa_state_path)
    
    #calculate the robust accuracy for 10 times, then report the average accuracy and std
    accs = []
    for i in range(10):
        adv_accuracy = clean_accuracy(
                                    model,
                                    x_adv,
                                    clean_y_test,
                                    batch_size=batch_size,
                                    device=device)
        accs.append(adv_accuracy)
    print(f'After auto attack, with {len(clean_x_test)} clean examples,  test for 10 times, robust accuracy: {np.mean(accs):.2%}±{np.std(accs):.2%}')
    print("===================================")
    # note: load adversarial images generated by our attack
    saved_adv_images = np.load(f"cifar100_adv.npy")
    saved_adv_images = torch.tensor(saved_adv_images).cuda()
    accs = []
    for i in range(10):
        adv_accuracy = clean_accuracy(
                                    model,
                                    saved_adv_images[:len(clean_y_test)],
                                    clean_y_test,
                                    batch_size=batch_size,
                                    device=device)
        accs.append(adv_accuracy)
    print("===================================")
    print(f'After our attack, with {len(clean_x_test)} clean examples,  test for 10 times, robust accuracy: {np.mean(accs):.2%}±{np.std(accs):.2%}')

In [12]:
wrapped_combined_model = torch.load('model_whole.pth')
data_dir = "data/"

time_start = time.time()
benchmark(
        wrapped_combined_model.eval(),
        dataset="cifar100",
        threat_model='Linf',
        device=torch.device("cuda"),
        eps=8/255,
        n_examples=100,  # test on 100 examples
        version='rand',
        batch_size=32,
        data_dir=data_dir,
    )
time_end = time.time()
print(f"Time cost:  {(time_end-time_start)/60} mins")

Files already downloaded and verified
With 100 clean examples, test for 10 times, clean accuracy: 62.40%±2.87%
setting parameters for rand version
using rand version including apgd-ce, apgd-dlr.
initial accuracy: 54.00%
apgd-ce - 1/2 - 11 out of 32 successfully perturbed
apgd-ce - 2/2 - 14 out of 22 successfully perturbed
robust accuracy after APGD-CE: 29.00% (total time 1371.9 s)
apgd-dlr - 1/1 - 3 out of 29 successfully perturbed
robust accuracy after APGD-DLR: 26.00% (total time 2221.5 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 26.00%
After auto attack, with 100 clean examples,  test for 10 times, robust accuracy: 48.50%±2.97%
After our attack, with 100 clean examples,  test for 10 times, robust accuracy: 9.70%±1.85%
Time cost:  37.4475524743398 mins
